In [1]:
from selenium import webdriver

In [2]:
import datetime, re, requests, io, time, random, string

In [3]:
from bs4 import BeautifulSoup

In [4]:
from credentials import email, password

In [5]:
driver = webdriver.Chrome('/Users/yoel/downloads/chromedriver')

In [6]:
# Go to wallmine.com
driver.get('https://wallmine.com')

In [7]:
if 'Make Smarter Investments' in driver.page_source:
    sign_in_link = driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a')
    sign_in_link.click()
    time.sleep(3)
    print('Going to sign in page')

Going to sign in page


In [8]:
if 'Sign in here, please.' in driver.page_source:
    login_email = driver.find_element_by_xpath('//*[@id="user_email"]')
    sign_in_password = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]')
    sign_in_password.click()
    time.sleep(0.2)

In [9]:
login_password = driver.find_element_by_xpath('//*[@id="user_password"]')
sign_in_button = driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button')

In [10]:
login_email.send_keys(email)
login_password.send_keys(password)
sign_in_button.click()
time.sleep(3)

In [11]:
if 'Stock market overview' in driver.page_source:
    print('On the main page')
    heatmap = driver.find_element_by_xpath('//*[@id="homepage-heatmap"]/a/div[2]')
    heatmap.click()
    time.sleep(2)

On the main page


In [12]:
overview_tab = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a')
overview_tab.click()
time.sleep(3)

In [13]:
if 'Free Stock Screener' in driver.page_source:
    print('here')
    stock_data = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]').text
stock_list = stock_data.split('\n')
stock_list

here


['▴ ▴',
 'Symbol',
 '▴ ▴',
 'Company',
 '▴ ▴',
 'Exchange',
 '▴ ▴',
 'Industry',
 '▴ ▴',
 'Market cap',
 '▴ ▴',
 'EBITDA',
 '▴ ▴',
 'P/E',
 '▴ ▴',
 'EV/EBITDA',
 '▴ ▴',
 'Debt/Equity',
 '▴ ▴',
 'Average volume',
 '▴ ▴',
 'Institutional ownership',
 '▴ ▴',
 'Earnings date',
 '▴ ▴',
 'Price',
 '▴ ▴',
 'Performance today',
 'AAPL Apple, Inc. NASDAQ Consumer Electronics $1.83T $74.542B 33.05 24.95 2.74 52.096M 62.12% 29 Oct 2020 $108.67 +1.45%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 34.61 22.40 1.55 34.993M 74.09% 22 Oct 2020 $203.19 +1.30%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.50T $30.457B 115.76 49.32 2.63 5.062M 57.69% 22 Oct 2020 $3,019.79 +0.66%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 28.63 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,428.29 +0.92%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 30.83 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,422.86 +0.96

In [14]:
stock_list.index('Performance today')

27

In [15]:
stock_list[28]

'AAPL Apple, Inc. NASDAQ Consumer Electronics $1.83T $74.542B 33.05 24.95 2.74 52.096M 62.12% 29 Oct 2020 $108.67 +1.45%'

In [16]:
stock_list[28].split(' $')
company_info_parsed = stock_list[28].split(' $')[0]
market_cap = stock_list[28].split(' $')[1]
market_info = stock_list[28].split(' $')[2]
price_info = stock_list[28].split(' $')[3].split(' ')
price = f'${price_info[0]}'
performance_today = price_info[1]
# print(company_info_parsed)
# price_info
# price

In [17]:
company_info = company_info_parsed.split(' ')
if 'NASDAQ' in company_info:
    idx = company_info.index('NASDAQ')
    exchange = company_info[idx]
    print('here')
elif 'NYSE' in company_info:
    idx = company_info.index('NYSE')
    exchange = company_info[idx]
elif 'NYSEMKT' in company_info:
    idx = company_info.index('NYSEMKT')
    exchange = company_info[idx]
    
industry = " ".join(company_info[idx + 1:])
symbol = company_info[0]
company_name = " ".join(company_info[1:idx])
company_name

# Currently parsed
# symbol, company_name, exchange, industry, market_cap, price, performance_today
# ebitda, p_e, ev_ebitda, debt_equity, average_volume, institutional_ownership, earnings_date

here


'Apple, Inc.'

In [18]:
# Parse market info
market_info.split(' ')

['74.542B', '33.05', '24.95', '2.74', '52.096M', '62.12%', '29', 'Oct', '2020']

In [19]:
ebitda = market_info.split(' ')[0]
p_e = market_info.split(' ')[1]
ev_ebitda = market_info.split(' ')[2]
debt_equity = market_info.split(' ')[3]
average_volume = market_info.split(' ')[4]
institutional_ownership = market_info.split(' ')[5]
earnings_date = " ".join(market_info.split(' ')[6:])
earnings_date

'29 Oct 2020'

In [20]:
x_stock = {
    "symbol": symbol,
    "company_name": company_name,
    "exchange": exchange,
    "industry": industry,
    "market_cap": market_cap,
    "price": price,
    "performance_today":  performance_today,
    "ebitda": ebitda,
    "p_e": p_e,
    "ev_ebitda": ev_ebitda,
    "debt_equity": debt_equity,
    "average_volume": average_volume,
    "institutional_ownership": institutional_ownership,
    "earnings_date": earnings_date
}

x_stock

{'symbol': 'AAPL',
 'company_name': 'Apple, Inc.',
 'exchange': 'NASDAQ',
 'industry': 'Consumer Electronics',
 'market_cap': '1.83T',
 'price': '$108.67',
 'performance_today': '+1.45%',
 'ebitda': '74.542B',
 'p_e': '33.05',
 'ev_ebitda': '24.95',
 'debt_equity': '2.74',
 'average_volume': '52.096M',
 'institutional_ownership': '62.12%',
 'earnings_date': '29 Oct 2020'}

In [28]:
for each_stock in all_stocks:
    print(parse_stock_data(each_stock))

here
{'symbol': 'AAPL', 'company_name': 'Apple, Inc.', 'exchange': 'NASDAQ', 'industry': 'Consumer Electronics', 'market_cap': '1.91T', 'price': '$106.90', 'performance_today': '-4.39%', 'ebitda': '74.542B', 'p_e': '32.59', 'ev_ebitda': '24.95', 'debt_equity': '2.74', 'average_volume': '52.096M', 'institutional_ownership': '62.12%', 'earnings_date': '29 Oct 2020'}
here
{'symbol': 'MSFT', 'company_name': 'Microsoft Corp.', 'exchange': 'NASDAQ', 'industry': 'Internet Services & Infrastructure', 'market_cap': '1.54T', 'price': '$200.88', 'performance_today': '-3.15%', 'ebitda': '65.259B', 'p_e': '34.60', 'ev_ebitda': '22.40', 'debt_equity': '1.55', 'average_volume': '34.993M', 'institutional_ownership': '74.09%', 'earnings_date': '22 Oct 2020'}
here
{'symbol': 'AMZN', 'company_name': 'Amazon.com, Inc.', 'exchange': 'NASDAQ', 'industry': 'Specialty Retail', 'market_cap': '1.51T', 'price': '$2,999.86', 'performance_today': '-4.13%', 'ebitda': '30.457B', 'p_e': '113.41', 'ev_ebitda': '49.32'

IndexError: list index out of range

In [24]:
all_stocks

['AAPL Apple, Inc. NASDAQ Consumer Electronics $1.83T $74.542B 33.05 24.95 2.74 52.096M 62.12% 29 Oct 2020 $108.67 +1.45%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 34.61 22.40 1.55 34.993M 74.09% 22 Oct 2020 $203.19 +1.30%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.50T $30.457B 115.76 49.32 2.63 5.062M 57.69% 22 Oct 2020 $3,019.79 +0.66%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 28.63 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,428.29 +0.92%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 30.83 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,422.86 +0.96%',
 'BABA Alibaba Group Holding Ltd. NYSE Specialty Retail $747.81B ¥19.774B 28.38 242.77 0.57 15.474M 48.80% 30 Oct 2020 $269.73 -1.18%',
 'FB Facebook, Inc. NASDAQ Internet Services & Infrastructure $725.94B $34.727B 30.18 19.46 0.32 26.764M 79.41% 28 Oct 2020 $249.53 +0.20%',
 'BRK-A Berkshire Hathaway Inc. NYSE Mult

In [61]:
index_list = []
final_list = []

for i in range(len(all_stocks)):
    each_stock = all_stocks[i]
    
    if 'Intel' in each_stock:
        final_list.append(each_stock)
    elif 'NT' in each_stock:
        pass
    elif '¥' in each_stock:
        pass
    elif '€' in each_stock:
        pass
    elif 'kr' in each_stock:
        pass
    elif 'XETRA' in each_stock:
        pass
    else:
        final_list.append(each_stock)
        
final_list

['AAPL Apple, Inc. NASDAQ Consumer Electronics $1.91T $74.542B 32.59 24.95 2.74 52.096M 62.12% 29 Oct 2020 $106.90 -4.39%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 34.60 22.40 1.55 34.993M 74.09% 22 Oct 2020 $200.88 -3.15%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.51T $30.457B 113.41 49.32 2.63 5.062M 57.69% 22 Oct 2020 $2,999.86 -4.13%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 28.57 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,415.21 -3.43%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $1.01T $47.254B 30.82 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,409.39 -3.45%',
 'FB Facebook, Inc. NASDAQ Internet Services & Infrastructure $725.94B $34.727B 30.88 19.46 0.32 26.764M 79.41% 28 Oct 2020 $249.02 -2.25%',
 'BRK-A Berkshire Hathaway Inc. NYSE Multi-line Insurance $520.31B $53.661B 19.54 11.11 0.92 1.012k 24.67% N/A $314,840.00 -1.80%',
 'BRK-B Berkshire Hathaway Inc. NYSE Multi-line I

In [25]:
final_list

NameError: name 'final_list' is not defined

In [26]:
x = parse_stock_data(final_list[1])

NameError: name 'final_list' is not defined

In [27]:
driver.get(f"https://wallmine.com/{x.get('exchange')}/{x.get('symbol')}")

NameError: name 'x' is not defined

In [28]:
for each_stock in final_list:
    x = parse_stock_data(each_stock)
    driver.get(f"https://wallmine.com/{x.get('exchange')}/{x.get('symbol')}")
    time.sleep(3)
    
    if x.get('symbol') in driver.page_source:
        print(f"We are on {x.get('company_name')} stock page")

NameError: name 'final_list' is not defined

In [29]:
final_list[16]

NameError: name 'final_list' is not defined

In [30]:
parse_stock_data(final_list[16])

NameError: name 'final_list' is not defined

In [44]:
def parse_stock_data(data):
    company_info_parsed = data.split(' $')[0]
    company_info = company_info_parsed.split(' ')
    if 'NASDAQ' in company_info:
        idx = company_info.index('NASDAQ')
        exchange = company_info[idx]
    elif 'NYSE' in company_info:
        idx = company_info.index('NYSE')
        exchange = company_info[idx]
    elif 'NYSEMKT' in company_info:
        idx = company_info.index('NYSEMKT')
        exchange = company_info[idx]
    industry = " ".join(company_info[idx + 1:])
    symbol = company_info[0]
    company_name = " ".join(company_info[1:idx])
    if 'N/A' in data.split(' $')[1]:
        market_cap_1 = data.split(' $')[1].split(' ')
        market_cap = market_cap_1[0]
        ebitda = market_cap_1[1]
        p_e = market_cap_1[2]
        ev_ebitda = market_cap_1[3]
        debt_equity = market_cap_1[4]
        average_volume = market_cap_1[5]
        institutional_ownership = market_cap_1[6]
        earnings_date = " ".join(market_cap_1[7:])
        price_info = data.split(' $')[2].split(' ')
        price = price_info[0]
        performance_today = price_info[1]
    else:
        market_cap = data.split(' $')[1]
        market_info = data.split(' $')[2]
        ebitda = market_info.split(' ')[0]
        p_e = market_info.split(' ')[1]
        ev_ebitda = market_info.split(' ')[2]
        debt_equity = market_info.split(' ')[3]
        average_volume = market_info.split(' ')[4]
        institutional_ownership = market_info.split(' ')[5]
        earnings_date = " ".join(market_info.split(' ')[6:])
        price_info = data.split(' $')[3].split(' ')
        price = price_info[0]
        performance_today = price_info[1]
    result = {
        "symbol": symbol,
        "company_name": company_name,
        "exchange": exchange,
        "industry": industry,
        "market_cap": market_cap,
        "price": price,
        "performance_today": performance_today,
        "ebitda": ebitda,
        "p_e": p_e,
        "ev_ebitda": ev_ebitda,
        "debt_equity": debt_equity,
        "average_volume": average_volume,
        "institutional_ownership": institutional_ownership, 
        "earnings_date": earnings_date
    }
    return result

In [45]:
final_list = []
for i in range(len(stock_list[28:])):
    each_stock = stock_list[28:][i]
    if 'Intel' in each_stock:
        final_list.append(each_stock)
    elif 'NT' in each_stock:
        pass
    elif '¥' in each_stock:
        pass
    elif '€' in each_stock:
        pass
    elif 'kr' in each_stock:
        pass
    elif 'XETRA' in each_stock:
        pass
    else:
        final_list.append(each_stock)

In [46]:
final_list

['AAPL Apple, Inc. NASDAQ Consumer Electronics $1.83T $74.542B 33.05 24.95 2.74 52.096M 62.12% 29 Oct 2020 $108.67 +1.45%',
 'MSFT Microsoft Corp. NASDAQ Internet Services & Infrastructure $1.54T $65.259B 34.61 22.40 1.55 34.993M 74.09% 22 Oct 2020 $203.19 +1.30%',
 'AMZN Amazon.com, Inc. NASDAQ Specialty Retail $1.50T $30.457B 115.76 49.32 2.63 5.062M 57.69% 22 Oct 2020 $3,019.79 +0.66%',
 'GOOG Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 28.63 18.82 0.37 1.791M 70.62% 22 Oct 2020 $1,428.29 +0.92%',
 'GOOGL Alphabet, Inc. NASDAQ Internet Services & Infrastructure $960.56B $47.254B 30.83 18.69 0.37 1.859M 82.29% 22 Oct 2020 $1,422.86 +0.96%',
 'FB Facebook, Inc. NASDAQ Internet Services & Infrastructure $725.94B $34.727B 30.18 19.46 0.32 26.764M 79.41% 28 Oct 2020 $249.53 +0.20%',
 'BRK-A Berkshire Hathaway Inc. NYSE Multi-line Insurance $520.31B $53.661B 19.19 11.11 0.92 1.012k 24.67% N/A $313,780.00 -0.34%',
 'BRK-B Berkshire Hathaway Inc. NYSE Multi-li

In [47]:
for each_stock in final_list:
    print(parse_stock_data(each_stock))
    time.sleep(1)

{'symbol': 'AAPL', 'company_name': 'Apple, Inc.', 'exchange': 'NASDAQ', 'industry': 'Consumer Electronics', 'market_cap': '1.83T', 'price': '108.67', 'performance_today': '+1.45%', 'ebitda': '74.542B', 'p_e': '33.05', 'ev_ebitda': '24.95', 'debt_equity': '2.74', 'average_volume': '52.096M', 'institutional_ownership': '62.12%', 'earnings_date': '29 Oct 2020'}
{'symbol': 'MSFT', 'company_name': 'Microsoft Corp.', 'exchange': 'NASDAQ', 'industry': 'Internet Services & Infrastructure', 'market_cap': '1.54T', 'price': '203.19', 'performance_today': '+1.30%', 'ebitda': '65.259B', 'p_e': '34.61', 'ev_ebitda': '22.40', 'debt_equity': '1.55', 'average_volume': '34.993M', 'institutional_ownership': '74.09%', 'earnings_date': '22 Oct 2020'}
{'symbol': 'AMZN', 'company_name': 'Amazon.com, Inc.', 'exchange': 'NASDAQ', 'industry': 'Specialty Retail', 'market_cap': '1.50T', 'price': '3,019.79', 'performance_today': '+0.66%', 'ebitda': '30.457B', 'p_e': '115.76', 'ev_ebitda': '49.32', 'debt_equity': '

{'symbol': 'BAC', 'company_name': 'Bank of America Corp.', 'exchange': 'NYSE', 'industry': 'Diversified Banks', 'market_cap': '219.63B', 'price': '23.34', 'performance_today': '+0.34%', 'ebitda': 'N/A', 'p_e': '11.08', 'ev_ebitda': 'N/A', 'debt_equity': '8.19', 'average_volume': '61.231M', 'institutional_ownership': '73.24%', 'earnings_date': '14 Oct 2020'}
{'symbol': 'KO', 'company_name': 'The Coca-Cola Co.', 'exchange': 'NYSE', 'industry': 'Soft Drinks', 'market_cap': '217.13B', 'price': '48.70', 'performance_today': '+1.02%', 'ebitda': '11.869B', 'p_e': '22.42', 'ev_ebitda': '21.04', 'debt_equity': '3.44', 'average_volume': '15.267M', 'institutional_ownership': '70.00%', 'earnings_date': '22 Oct 2020'}
{'symbol': 'MRK', 'company_name': 'Merck & Co., Inc.', 'exchange': 'NYSE', 'industry': 'Pharmaceuticals', 'market_cap': '216.43B', 'price': '83.15', 'performance_today': '+0.63%', 'ebitda': '16.985B', 'p_e': '20.01', 'ev_ebitda': '13.77', 'debt_equity': '2.25', 'average_volume': '8.02